# Oscillation increases the precision of network firing 

In [1]:
"""Oscillation increases the precision of network firing"""

from __future__ import division

import csv

import numpy as np
from docopt import docopt

from fakespikes import util as fsutil
from platypus.algorithms import NSGAII
from platypus.core import Problem
from platypus.types import Real
from voltagebudget.neurons import lif, readout
from voltagebudget.util import k_spikes
from voltagebudget.util import estimate_communication
from voltagebudget.util import estimate_computation
from voltagebudget.util import mean_budget
from voltagebudget.util import filter_budget
from voltagebudget.util import filter_spikes
from voltagebudget.util import filter_times

from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import Range1d
output_notebook()

import scipy
import numpy as np

from fakespikes import util as fsutil

Loading BokehJS ...

In [2]:
seed = 10
# ---------------------------------------------------------------------
# Create input
t = 0.3
t_stim = 0.1
time_step = 1e-4

k = 20
w = 1e-4
a = 10000

ns, ts = k_spikes(t_stim, k, w, a=a, dt=time_step, seed=42)
times = fsutil.create_times(t, time_step)

# ---------------------------------------------------------------------
# Create Y population
N = 50

# Osc params
f = 30
A = 0.01/2
phi = 0

# Comp params
w_in = 0.1e-9
sigma_in = 10
w_y = [w_in, w_in * sigma_in]
bias = [5e-3, 5e-3/10]


# Create Y, then Z
ns_y, ts_y, vs_y = lif(t,
                       N,
                       ns,
                       ts,
                       w_in=[w_in, w_in * sigma_in],
                       bias=bias,
                       f=f,
                       A=A,
                       phi=phi,
                       r_b=0,
                       budget=True,
                       report=None,
                       time_step=time_step, 
                       seed=seed)

print(vs_y.keys())

TypeError: k_spikes() got an unexpected keyword argument 'a'

In [ ]:
# ---------------------------------------------------------------------
# Create input
t = 0.3
t_stim = 0.1
time_step = 1e-4

k = 20
w = 1e-4
a = 10000

ns, ts = k_spikes(t_stim, k, w, a=a, dt=time_step, seed=42)
times = fsutil.create_times(t, time_step)

# ---------------------------------------------------------------------
# Create Y population
N = 50

# Osc params
f = 30
A = 0.01
phi = 0.012

# Comp params
w_in = 0.1e-9
sigma_in = 10
w_y = [w_in, w_in * sigma_in]
bias = [5e-3, 5e-3/10]


# Create Y, then Z
ns_o, ts_o, vs_o = lif(t,
                       N,
                       ns,
                       ts,
                       w_in=[w_in, w_in * sigma_in],
                       bias=bias,
                       f=f,
                       A=A,
                       phi=phi,
                       r_b=0,
                       budget=True,
                       report=None,
                       time_step=time_step,
                       seed=seed)

print(vs_y.keys())

In [ ]:
stim_window = [0.1, 0.12]
ns_y, ts_y = filter_spikes(ns_y, ts_y, stim_window)
ns_o, ts_o = filter_spikes(ns_o, ts_o, stim_window)

p = figure(plot_width=400, plot_height=200)
p.circle(ts, ns + 2*N, color="black", size=1.0)
p.circle(ts_y, ns_y + N, color="grey", size=1.0)
p.circle(ts_o, ns_o, color="red", size=1.0)
p.x_range = Range1d(0.095, 0.2)
p.grid.grid_line_color = None
show(p)

In [ ]:
ts_yr, ns_yr, vs_yr = readout(t, ns_y, ts_y, w_read=2e-9 / N)
ts_or, ns_or, vs_or = readout(t, ns_o, ts_o, w_read=2e-9 / N)

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.line(times, vs_yr['vm'][0,:], color='grey')
p.line(times, vs_or['vm'][0,:], color='red')

p.x_range = Range1d(0.095, 0.2)
p.y_range = Range1d(-65e-3, -52e-3)

p.grid.grid_line_color = None
show(p)